<a href="https://colab.research.google.com/github/FatimA-2/Data-Science-Statistics-and-Machine-Learning/blob/master/smld_(resampling_methods)_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resampling Methods

In this lab, we explore the resampling techniques covered in the part on Resampling Methods. We begin by placing most of our imports at this top level.

You will need to install the `ISLP` package, which provides access to the datasets and custom-built functions. This also installs most other packages needed in the labs. The Python resources page has a link to the ISLP documentation website.

In [ ]:
%reset -f
!pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
  

In [ ]:
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import (cross_validate, KFold, train_test_split)

In [ ]:
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, sklearn_sm, poly)

## The Validation Set Approach
We explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the  `Auto`  data set.

In [ ]:
Auto = load_data('Auto')
Auto.head(10)


We use the function `train_test_split()` to split the data into training and validation sets. As there are 392 observations, we split into two equal sets of size 196 using the argument `test_size=196`. It is generally a good idea to set a random seed when performing operations like this that contain an element of randomness, so that the results obtained can be reproduced precisely at a later time. We set the random seed of the splitter with the argument `random_state=0`.

In [ ]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

Now we can fit a linear regression using only the observations corresponding to the training set `Auto_train`. We will use `ModelSpec()` (renamed `MS()` in the preamble) to create an object, which will then be used to construct the corresponding model matrix. For example, if we want to add "horsepower" as a predictor in our linear model, we will write:



In [ ]:
X = MS(['horsepower'])
X_train = X.fit_transform(Auto_train)
X_train.head(10)

`X_train` is the training model matrix, which includes a column of ones, necessary for estimating the intercept of the linear model, and a column associated with the predictor "horsepower", necessary for estimating the corresponding coefficient of the linear model.

Let's prepare the training model vector for the response variable "mpg":

In [ ]:
y_train = Auto_train['mpg'] #mpg stands for "miles per gallon": it indicates how many miles we can travel on a gallon
y_train.head(10)

Now we can estimate our linear model:

In [ ]:
model = sm.OLS(y_train, X_train)
results = model.fit()
print(results.summary())

We now use the `predict()` method of `results` evaluated on the training model matrix using the validation data set. First, we need to create the validation model matrix and the validation model vector for the response:

In [ ]:
X_valid = X.transform(Auto_valid)
y_valid = Auto_valid['mpg']
y_valid.head(10)

Then, we can make predictions for the validation set using the results obtained from the training set

In [ ]:
y_valid_pred = results.predict(X_valid)
y_valid_pred.head(10)

The validation MSE of our model will be:

In [ ]:
np.mean((y_valid - y_valid_pred)**2)

Hence our estimate for the validation MSE of  the linear regression fit is $23.62$.

We can also estimate the validation error for higher-degree polynomial regressions. We first provide a function `evalMSE()` that takes as input a string representing the model as well as a training and validation set and returns the MSE on the validation set.

In [ ]:
def evalMSE(predictors, response, train, valid):

   X = MS(predictors)
   X_train = X.fit_transform(train)
   y_train = train[response]

   X_valid = X.transform(valid)
   y_valid = valid[response]

   results = sm.OLS(y_train, X_train).fit()
   y_valid_pred = results.predict(X_valid)

   return np.mean((y_valid - y_valid_pred)**2)

Let’s use this function to estimate the validation MSE using linear, quadratic and cubic fits. We use the `enumerate()` function here, which gives both the values and indices of objects as one iterates over a for loop. Moreover, the `poly()` function supplied in `ISLP` specifies that columns representing polynomial functions of its first argument are added to the model matrix.

In [ ]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

These error rates for the linear, quadratic and cubic fits are $23.62, 18.76$, and $18.80$, respectively.

If we choose a different training/validation split instead, then we can expect somewhat different errors on the validation set. For example, let's consider a random seed for the splitter with argument `random_state=7` instead of `random_state=0`:

In [ ]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=7)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

Using this split of the observations into a training set and a validation set, we find that the validation set error rates for the models with linear, quadratic, and cubic terms are $25.47$, $20.01$, and $19.96$, respectively.

In [ ]:
%reset -f
!pip install ISLP
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import (cross_validate, KFold, train_test_split)
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, sklearn_sm, poly)

## Cross-Validation
The simplest way to cross-validate in Python is to use `sklearn`, which has a different Application Programming Interface (API) than `statsmodels`, the library we have been using to fit generalized linear models (GLMs).

This is a problem which often confronts data scientists: "I have a function to do task $A$ (for example estimating GLMs with `statsmodels`), and need to feed it into something that performs task $B$ (for example cross-validating with `sklearn`), so that I can compute $B(A(D))$, where $D$ is my data." When $A$ and $B$ don’t naturally speak to each other, this requires the use of a *wrapper*.

The `ISLP` package provides a wrapper, `sklearn_sm()`, that enables us to easily use the cross-validation tools of `sklearn` with models fit by `statsmodels`.

The class `sklearn_sm()` has  as its first argument a model from `statsmodels`. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when fitting the model. For example, to fit a logistic regression model we have to specify a `family` argument. This
is passed as `model_args={'family':sm.families.Binomial()}`.

Here is our wrapper in action:

In [ ]:
Auto = load_data('Auto')
predictors = ['horsepower']
model = sklearn_sm(sm.OLS, MS(predictors))
X, Y = Auto[predictors], Auto['mpg']

Now, we will perform cross-validation using `cross_validate()`. The function takes the model to be estimated, the predictors, the response variable, and the argument `cv`, which specifies the details of the cross-validation folds. In particular, we used the `KFold()` function for the argument `cv`, which partitions the data into folds. Because we selected a number of folds equal to the total number of observations (i.e. `n_splits=len(Auto)`), we effectively performed leave-one-out cross-validation (LOOCV).

In [ ]:
cv_folds = KFold(n_splits=len(Auto), shuffle=True, random_state=0) #LOOCV is not affected by the random_state!
cv_results = cross_validate(model, X, Y, cv=cv_folds)
print("Validation MSE for each fold:")
print(cv_results['test_score'])
print("Number of folds: " + str(len(cv_results['test_score'])))

The LOOCV estimate for the test MSE is the average of these 392 test error estimates:

In [ ]:
cv_err = np.mean(cv_results['test_score'])
cv_err #equal to 24.23

We can repeat this procedure for increasingly complex polynomial fits. To automate the process, we again use a for loop which iteratively fits polynomial regressions of degree 1 to 3, computes the associated cross-validation error, and stores it in the $i$th element of the vector `cv_err`. The variable `d` in the for loop corresponds to the degree of the polynomial.

In [ ]:
cv_err = np.zeros(3)
cv_folds = KFold(n_splits=len(Auto), shuffle=True, random_state=0) #LOOCV is not affected by the random_state!
predictors = np.array(Auto['horsepower'])
model = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1,4)):
    X = np.power.outer(predictors, np.arange(d+1))
    cv_results = cross_validate(model, X, Y, cv=cv_folds)
    cv_err[i] = np.mean(cv_results['test_score'])
cv_err

Above we introduced `np.power.outer(a, b)`. It is a NumPy function that computes the outer product of powers. In other words, it raises each element of `a` to each element of `b`.

`np.arange(d+1)` creates an array of integers from 0 to `d`. For example, if `d` = 3, the resulting array will be [0, 1, 2, 3]. So, the outer product of powers will be: $predictor^0 = 1$, $predictor^1 = predictor$, $predictor^2$, $predictor^3$

In the CV above (LOOCV), we set the number of folds equal to $n$, where $n$ is the number of observations. Of course we can also use a number of folds less than $n$. The code is very similar to the above (and is significantly faster). Here we use `KFold()` to partition the data into $10$ random folds. We use `random_state` to set a random seed and initialize a vector `cv_err` in which we will store the CV errors corresponding to the polynomial fits of degrees one to three.

In [ ]:
cv_err = np.zeros(3)
cv_folds = KFold(n_splits=10, shuffle=True, random_state=0) #k-Fold is affected by the random_state when k < n!
predictors = np.array(Auto['horsepower'])
model = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1,4)):
    X = np.power.outer(predictors, np.arange(d+1))
    cv_results = cross_validate(model, X, Y, cv=cv_folds)
    cv_err[i] = np.mean(cv_results['test_score'])
cv_err

In [ ]:
%reset -f
!pip install ISLP
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import (cross_validate, KFold, train_test_split)
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, sklearn_sm, poly)

## The Bootstrap
We illustrate the use of the bootstrap in the simple example where we supposed that we wish to invest a fixed sum of money in two financial
assets that yield returns of $X$ and $Y$. The goal is to estimate the
sampling variance of the parameter $\alpha$

After loading the `Portfolio` dataset from the `ISLP` package, we will
create a function `alpha_func()`, which takes as input a dataframe `D` assumed to have columns `X` and `Y`, as well as a vector `idx` indicating which observations should be used to estimate
$\alpha$. The function then outputs the estimate for $\alpha$ based on
the selected observations.

In [ ]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
   cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
   return ((cov_[1,1]-cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))
Portfolio

Since the observations in the dataset are indexed with an index ranging from 0 to 99, if we want to estimate $\alpha$ considering all the observations in the dataset, we can use `np.arange(100)` in the `idx` argument. Indeed, `np.arange(100)` will return an array of values ranging from 0 to 99

In [ ]:
np.arange(100)

Let's estimate $\alpha$ using all 100 observations

In [ ]:
alpha_func(Portfolio, np.arange(100))

From the `Portfolio` dataset, we can draw bootstrap datasets, whose characteristic is that observations are sampled from the original dataset with replacement. We can perform sampling with replacement using a random number generator (RNG) with `np.random.default_rng(0)`. In particular

In [ ]:
rng = np.random.default_rng(0) #random number generator with seed 0. This makes the random number generator reproducible
rng.choice(np.arange(100), 100, replace=True)

`rng.choice(np.arange(100), 100, replace=True)` returned an array of 100 pseudorandom numbers (as specified by the second argument, `100`), selected from the range 0 to 99 (as defined by the first argument, `np.arange(100)`), with replacement (`replace=True`). These numbers represent the indices of the observations from the original dataset that will be included in the bootstrap dataset

Let's estimate the bootstrap standard error of $\hat{\alpha}$. We can do this by creating a simple function, `boot_SE()`

In [ ]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first, second = 0, 0 #values used to calculate the SE bootstrap
    n = n or len(D) #if the third argument of the function is None, n = len(D); if the argument is n, n = n
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first += value
        second += value**2
    return np.sqrt(second / B - (first / B)**2) #square_root(E(alpha^2) - (E(alpha))^2)

    #Note that E(alpha^2) - (E(alpha))^2 is an alternative formula for the variance, i.e.:
    #Var(alpha) = E(alpha - E(alpha))^2 = E(alpha^2) - (E(alpha))^2

Notice the use of `_` as a loop variable in `for _ in range(B)`. This is often used if the value of the counter is unimportant and simply makes sure  the loop is executed `B` times.

Let’s use our function to evaluate the accuracy of our estimate of $\alpha$ using $B=1000$ bootstrap replications.

In [ ]:
alpha_SE = boot_SE(func=alpha_func, D=Portfolio, B=1000, seed=0)
alpha_SE

The final output shows that the bootstrap estimate for ${\rm SE}(\hat{\alpha})$ is $0.0912$.

##References
This Jupyter Notebook is a revised version curated by Paolo Mustica, based on the original Notebook provided by the authors James, G., Witten, D., Hastie, T., Tibshirani, R., and Taylor, J. (2023) in An Introduction to Statistical Learning with Applications in Python (Chapter 5), published by Springer Nature.